#Install

In [3]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
import warnings
import random
warnings.filterwarnings("ignore")

In [4]:
!pip install pytorch-tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 443 kB/s 


In [5]:
import torch

from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier

from sklearn.metrics import f1_score, accuracy_score

In [6]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "-1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
device='cuda' if torch.cuda.is_available() else 'CPU'

if device=='cuda':
  torch.cuda.manual_seed_all(0)

In [8]:
import numpy as np
np.random.seed(0)

#Data Processing

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
base_path = '/content/drive/MyDrive/2022_Unithon_Data/data/dataset'

In [11]:
# 파일 불러오기
def dataframe_from_csv(target):
  return pd.read_csv(target).rename(columns=lambda x:x.strip())

def dataframe_from_csvs(targets):
  return pd.concat([dataframe_from_csv(x) for x in targets])

train_files = sorted([x for x in Path(base_path+'/train/').glob('*.csv')])
val_files = sorted([x for x in Path(base_path+'/val/').glob('*.csv')])

train = dataframe_from_csvs(train_files)
val = dataframe_from_csvs(val_files)
test = pd.read_csv(base_path+'/test.csv')
print(f'train: {len(train)}')
print(f'validation: {len(val)}')
print(f'test: {len(test)}')

train: 62564
validation: 7820
test: 7820


In [12]:
train = train.sample(frac=1).reset_index(drop=True)
train

,site,sid,leaktype,C01,C02,C03,C04,C05,C06,C07,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,S-5011010101,S-0359369084272844,in,0,780,37,125,49,6,31,...,3,0,3,0,0,0,0,3,0,310
1,S-4677025033,S-0359369083950515,in,7,74,17,17,22,17,13,...,7,9,13,9,9,11,9,11,11,400
2,S-4677025028,S-0359369084289301,in,17,138,40,34,24,24,10,...,0,0,0,0,0,0,3,0,3,280
3,S-2911010107,S-0359369084032750,normal,5,128,5,5,6,7,6,...,6,6,7,6,7,6,6,7,6,260
4,S-3611010100,S-0035514005601221,out,6,98,12,18,18,10,6,...,4,4,6,4,4,4,4,4,4,420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62559,S-4571025029,S-0359369084049200,normal,6,20,6,10,7,8,6,...,8,8,10,8,8,13,8,23,11,260
62560,S-4571025029,S-0359369084037072,normal,4,23,6,6,5,5,4,...,5,5,5,4,5,5,6,5,5,270
62561,S-4673025027,S-0359369085161442,other,0,95,13,3,3,10,3,...,3,3,3,3,3,3,3,3,3,280
62562,S-2911010107,S-0359369084050547,normal,4,11,6,6,6,6,6,...,6,6,6,6,6,6,7,7,6,260


In [13]:
train = train.drop(['site', 'sid'], axis=1)
val = val.drop(['site', 'sid'], axis=1)
test = test.drop(['site', 'sid'], axis=1)
train['leaktype'].replace(['out','in','noise','other','normal'], [0,1,2,3,4], inplace=True)
val['leaktype'].replace(['out','in','noise','other','normal'], [0,1,2,3,4], inplace=True)
test['leaktype']=""

# # train을 target과 feature로 나눠줍니다.
train_x=train.drop(['leaktype'], axis=1)
train_y=train['leaktype']
val_x=val.drop(['leaktype'], axis=1)
val_y=val['leaktype']
test_x=test.drop(['leaktype'], axis=1)
test_y=test['leaktype']

In [14]:
print(train_x)
print(train_y)

       C01  C02  C03  C04  C05  C06  C07  C08  C09  C10  ...  C17  C18  C19  \
0        0  780   37  125   49    6   31    6    6    0  ...    3    0    3   
1        7   74   17   17   22   17   13   13   13   17  ...    7    9   13   
2       17  138   40   34   24   24   10   13   13   10  ...    0    0    0   
3        5  128    5    5    6    7    6    6    6    6  ...    6    6    7   
4        6   98   12   18   18   10    6    4    6    6  ...    4    4    6   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
62559    6   20    6   10    7    8    6    7    7    6  ...    8    8   10   
62560    4   23    6    6    5    5    4    5    5    5  ...    5    5    5   
62561    0   95   13    3    3   10    3    3    3    3  ...    3    3    3   
62562    4   11    6    6    6    6    6    7    6    6  ...    6    6    6   
62563    2   80    5    4    5    4    4    4    4    4  ...    4    4    4   

       C20  C21  C22  C23  C24  C25  C26  
0       

In [15]:
train

,leaktype,C01,C02,C03,C04,C05,C06,C07,C08,C09,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1,0,780,37,125,49,6,31,6,6,...,3,0,3,0,0,0,0,3,0,310
1,1,7,74,17,17,22,17,13,13,13,...,7,9,13,9,9,11,9,11,11,400
2,1,17,138,40,34,24,24,10,13,13,...,0,0,0,0,0,0,3,0,3,280
3,4,5,128,5,5,6,7,6,6,6,...,6,6,7,6,7,6,6,7,6,260
4,0,6,98,12,18,18,10,6,4,6,...,4,4,6,4,4,4,4,4,4,420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62559,4,6,20,6,10,7,8,6,7,7,...,8,8,10,8,8,13,8,23,11,260
62560,4,4,23,6,6,5,5,4,5,5,...,5,5,5,4,5,5,6,5,5,270
62561,3,0,95,13,3,3,10,3,3,3,...,3,3,3,3,3,3,3,3,3,280
62562,4,4,11,6,6,6,6,6,7,6,...,6,6,6,6,6,6,7,7,6,260


In [16]:
nunique = train.nunique()
types = train.dtypes

print(nunique)
print(types)

leaktype      5
C01          89
C02         768
C03         173
C04         120
C05         162
C06         122
C07          78
C08          81
C09          83
C10          72
C11         153
C12         154
C13         195
C14         217
C15         132
C16         114
C17         119
C18         170
C19         253
C20         278
C21         259
C22         239
C23         249
C24         191
C25          91
C26         451
dtype: int64
leaktype    int64
C01         int64
C02         int64
C03         int64
C04         int64
C05         int64
C06         int64
C07         int64
C08         int64
C09         int64
C10         int64
C11         int64
C12         int64
C13         int64
C14         int64
C15         int64
C16         int64
C17         int64
C18         int64
C19         int64
C20         int64
C21         int64
C22         int64
C23         int64
C24         int64
C25         int64
C26         int64
dtype: object


In [17]:
categorical_columns = []
categorical_dims =  {}
for col in train.columns:
  print(col)
  categorical_columns.append(col)
  categorical_dims[col] = nunique[col]

leaktype
C01
C02
C03
C04
C05
C06
C07
C08
C09
C10
C11
C12
C13
C14
C15
C16
C17
C18
C19
C20
C21
C22
C23
C24
C25
C26


In [18]:
print(categorical_columns)
print(categorical_dims)

['leaktype', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26']
{'leaktype': 5, 'C01': 89, 'C02': 768, 'C03': 173, 'C04': 120, 'C05': 162, 'C06': 122, 'C07': 78, 'C08': 81, 'C09': 83, 'C10': 72, 'C11': 153, 'C12': 154, 'C13': 195, 'C14': 217, 'C15': 132, 'C16': 114, 'C17': 119, 'C18': 170, 'C19': 253, 'C20': 278, 'C21': 259, 'C22': 239, 'C23': 249, 'C24': 191, 'C25': 91, 'C26': 451}


In [19]:
target = 'leaktype'

In [20]:
unused_feat = []

features = [ col for col in train.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

#TabNet Model

In [159]:
model = TabNetClassifier(
    n_d=64, n_a=64, n_steps=5,
    gamma=1.5, n_independent=2, n_shared=2,
    cat_idxs=[],
    cat_dims=[],
    cat_emb_dim=1,
    lambda_sparse=1e-4, momentum=0.3, clip_value=2.,
    # optimizer = torch.optim.SGD(classifier.parameters(), lr=0.1, momentum=0.9),
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=2, eta_min=0),
    # optimizer_fn = torch.optim.SGD,
    # optimizer_params=dict(lr=0.02, momentum=0.9),
    optimizer_fn=torch.optim.Adam,
    # optimizer_params=dict(lr=2e-2),
    scheduler_params = {"gamma": 0.5,
                     "step_size": 50},
    scheduler_fn=torch.optim.lr_scheduler.StepLR, 
    # scheduler_fn = torch.optim.lr_scheduler.CosineAnnealingLR,
    # scheduler_params = {"T_max":100, "eta_min":0.001},
    epsilon=1e-15,
    device_name=device,
    # device_name='cpu',
    # mask_type="entmax"
    verbose=1
)

In [20]:
len(cat_idxs)

26

In [78]:
cat_dims

[89,
 768,
 173,
 120,
 162,
 122,
 78,
 81,
 83,
 72,
 153,
 154,
 195,
 217,
 132,
 114,
 119,
 170,
 253,
 278,
 259,
 239,
 249,
 191,
 91,
 451]

#Train

In [22]:
len(train_x)

62564

In [100]:
X_train = train[features].values
y_train = train_y.values

X_valid = val_x[features].values
X_test = test_x[features].values

y_valid = val_y.values
y_test = test_y.values

In [72]:
X_train.shape

(62564, 26)

In [73]:
y_train.shape

(62564,)

In [74]:
X_valid.shape

(7820, 26)

In [75]:
y_valid.shape

(7820,)

In [119]:
max_epochs = 2500

In [134]:
from pytorch_tabnet.augmentations import ClassificationSMOTE
aug = ClassificationSMOTE(p=0.2)

In [160]:
model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    max_epochs=max_epochs, patience=100,
    batch_size=16384, virtual_batch_size=64,
    augmentations=aug    
) 

epoch 0  | loss: 2.82712 | train_accuracy: 0.22315 | valid_accuracy: 0.22276 |  0:00:04s
epoch 1  | loss: 2.00809 | train_accuracy: 0.20638 | valid_accuracy: 0.2133  |  0:00:08s
epoch 2  | loss: 1.72244 | train_accuracy: 0.20967 | valid_accuracy: 0.20742 |  0:00:13s
epoch 3  | loss: 1.53799 | train_accuracy: 0.2523  | valid_accuracy: 0.25396 |  0:00:17s
epoch 4  | loss: 1.47726 | train_accuracy: 0.19722 | valid_accuracy: 0.2009  |  0:00:21s
epoch 5  | loss: 1.424   | train_accuracy: 0.2095  | valid_accuracy: 0.20601 |  0:00:26s
epoch 6  | loss: 1.42759 | train_accuracy: 0.34824 | valid_accuracy: 0.35831 |  0:00:30s
epoch 7  | loss: 1.36483 | train_accuracy: 0.23828 | valid_accuracy: 0.23645 |  0:00:35s
epoch 8  | loss: 1.33201 | train_accuracy: 0.34697 | valid_accuracy: 0.34629 |  0:00:39s
epoch 9  | loss: 1.32738 | train_accuracy: 0.22607 | valid_accuracy: 0.22481 |  0:00:43s


KeyboardInterrupt: ignored

#Validate

In [ ]:
val_pred = model.predict(val_x)
print(val_pred)

[4 0 0 ... 4 4 4]


In [ ]:
print(val_y)

0       0
1       0
2       0
3       0
4       0
       ..
2457    4
2458    4
2459    4
2460    4
2461    4
Name: leaktype, Length: 7820, dtype: int64


In [ ]:
print("Validation F1 score: ", f1_score(val_y, val_pred, average='macro'))

Validation F1 score:  0.6369076286695308


In [ ]:
print("Validation Accuracy Score: ", accuracy_score(val_y, val_pred))

Validation Accuracy Score:  0.7112531969309462


#Test